In [5]:
# Part I - IoT sensor data encryption simulation
import json
import base64
import secrets
import time
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from datetime import datetime
import random

# ---------------------------------------------------------
# Function to generate IoT sensor readings (temperature + humidity + timestamp)
# ---------------------------------------------------------
def generate_sensor_reading():
    reading = {
        "timestamp": datetime.utcnow().isoformat() + "Z",   # Reading timestamp in UTC
        "temperature_C": round(random.uniform(15.0, 35.0), 2),  # Random temperature value
        "humidity_pct": round(random.uniform(20.0, 90.0), 2),   # Random humidity value
        "device_id": "sensor-001"   # Device ID
    }
    return reading


# =========================================================
#                🔐 AES-GCM encryption functions
# =========================================================

# ---------------------------------------------------------
# Function to encrypt data using AES-GCM
# ---------------------------------------------------------
def encrypt_aes_gcm(key, plaintext_bytes):

    # nonce: random value used to ensure encryption uniqueness
    nonce = get_random_bytes(12)  # GCM recommended 12-byte nonce

    # Create AES-GCM cipher object
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

    # Encrypt + generate authentication tag (to detect tampering)
    ciphertext, tag = cipher.encrypt_and_digest(plaintext_bytes)

    # Encode values in Base64 for safe transmission
    return {
        "nonce": base64.b64encode(nonce).decode(),
        "ciphertext": base64.b64encode(ciphertext).decode(),
        "tag": base64.b64encode(tag).decode()
    }


# ---------------------------------------------------------
# Function to decrypt AES-GCM encrypted data
# ---------------------------------------------------------
def decrypt_aes_gcm(key, nonce_b64, ciphertext_b64, tag_b64):

    # Decode Base64 values back to original bytes
    nonce = base64.b64decode(nonce_b64)
    ciphertext = base64.b64decode(ciphertext_b64)
    tag = base64.b64decode(tag_b64)

    # Create AES-GCM cipher object
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

    # Attempt decryption + tag verification
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)

    return plaintext


# =========================================================
#            🔥 Device-side simulation (Sender)
# =========================================================

# Generate encryption key (AES-256 = 32 bytes)
key = get_random_bytes(32)
print("Device symmetric key (base64) - store securely (example):",
      base64.b64encode(key).decode())

# Generate sensor reading
reading = generate_sensor_reading()
reading_json = json.dumps(reading).encode()

print("\n--- Plain sensor reading ---")
print(reading)

# Encrypt reading
enc = encrypt_aes_gcm(key, reading_json)

print("\n--- Encrypted payload (to transmit) ---")
print(json.dumps(enc, indent=2))


# =========================================================
#                 🔓 Receiver-side simulation
# =========================================================
print("\n--- Receiver: decrypting payload ---")

# Decrypt
decrypted_bytes = decrypt_aes_gcm(
    key,
    enc["nonce"],
    enc["ciphertext"],
    enc["tag"]
)

decrypted_json = json.loads(decrypted_bytes.decode())

print("Decrypted reading:")
print(decrypted_json)


# =========================================================
#                  🔥 Tampering detection test
# =========================================================
print("\n--- Tampering test (modify ciphertext) ---")

tampered = dict(enc)

# Flip the first byte in ciphertext → should cause failure
ct = bytearray(base64.b64decode(enc["ciphertext"]))
ct[0] ^= 1   # Flip the first bit
tampered["ciphertext"] = base64.b64encode(bytes(ct)).decode()

try:
    decrypt_aes_gcm(key, tampered["nonce"], tampered["ciphertext"], tampered["tag"])
    print("Tampering not detected (unexpected)")
except Exception as e:
    print("Tampering detected during decryption:", str(e))


Device symmetric key (base64) - store securely (example): 7K834q8MDAxB0RU6VZSCpjtOiYdPwpbtjx44zyjnKV0=

--- Plain sensor reading ---
{'timestamp': '2025-11-26T14:52:39.709944Z', 'temperature_C': 16.61, 'humidity_pct': 59.71, 'device_id': 'sensor-001'}

--- Encrypted payload (to transmit) ---
{
  "nonce": "hStqjaYiAQ2er2K6",
  "ciphertext": "HhiRCQHz9Yq+CAYjlmwGS2cm8oNSlsjUUwZTryZhzcGVbNJcjRi3kL4YuFSV2lpAPKmPh3SsriXL6yRtg0xxzLHSsijDn+MnuzqAR2q5p3eO2G5okQoTKX2nrL2nnSG3ERAUs13yIUNY9nHBlOCBO8AEQjXomw==",
  "tag": "qLjyqEGrwU90KsRRPY8eCg=="
}

--- Receiver: decrypting payload ---
Decrypted reading:
{'timestamp': '2025-11-26T14:52:39.709944Z', 'temperature_C': 16.61, 'humidity_pct': 59.71, 'device_id': 'sensor-001'}

--- Tampering test (modify ciphertext) ---
Tampering detected during decryption: MAC check failed


/tmp/ipython-input-280331162.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z",   # Reading timestamp in UTC
